In [1]:
#import libraries

import requests
from bs4 import BeautifulSoup
import glob
from datetime import datetime
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import calmap
%matplotlib inline

In [2]:
# Ministry of Health and Family Welfare url
URL = 'https://www.mohfw.gov.in'
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

html_thead = soup.find_all('thead')[-1]
html_th = [tr for tr in html_thead.find_all('tr')]

html_tbody = soup.find_all('tbody')[-1]
html_text = [tr for tr in html_tbody.find_all('tr')]

headings = []

for tr in html_th:
    th = tr.find_all(['th'])
    row = [i.text for i in th]
    headings.append(row)

print(headings)

content = []
for tr in html_text:
    td = tr.find_all(['td'])
    row = [i.text for i in td]
    content.append(row)
    
#print(content)

[['S. No.', 'Name of State / UT', 'Active Cases*', 'Cured/Discharged/Migrated*', 'Deaths**', 'Total Confirmed cases*']]


In [3]:
data= pd.DataFrame(content[:-6], columns=headings[0])

In [4]:
data

S. No.                        Name of State / UT Active Cases*  \
0       1               Andaman and Nicobar Islands            11   
1       2                            Andhra Pradesh          3340   
2       3                         Arunachal Pradesh            92   
3       4                                     Assam          1955   
4       5                                     Bihar          1972   
5       6                                Chandigarh            59   
6       7                              Chhattisgarh           674   
7       8  Dadra and Nagar Haveli and Daman and Diu            45   
8       9                                     Delhi         27741   
9      10                                       Goa           560   
10     11                                   Gujarat          6103   
11     12                                   Haryana          4750   
12     13                          Himachal Pradesh           189   
13     14                         Jammu and Kashmir          2427   
14     15                                 Jharkhand           734   
15     16                                 Karnataka          2828   
16     17                                    Kerala          1351   
17     18                                    Ladakh           594   
18     19                            Madhya Pradesh          2374   
19     20                               Maharashtra         51935   
20     21                                   Manipur           360   
21     22                                 Meghalaya            13   
22     23                                   Mizoram           120   
23     24                                  Nagaland            90   
24     25                                    Odisha          1280   
25     26                                Puducherry           129   
26     27                                    Punjab           881   
27     28                                 Rajasthan          2762   
28     29                                    Sikkim            66   
29     30                                Tamil Nadu         21993   
30     31                                 Telangana          2412   
31     32                                   Tripura           578   
32     33                               Uttarakhand           743   
33     34                             Uttar Pradesh          5259   
34     35                               West Bengal          5261   

   Cured/Discharged/Migrated* Deaths** Total Confirmed cases*  
0                          33        0                     44  
1                        3641       90                   7071  
2                           7        0                     99  
3                        2642        8                   4605  
4                        4926       44                   6942  
5                         303        6                    368  
6                        1180       10                   1864  
7                          12        0                     57  
8                       17457     1904                  47102  
9                          96        0                    656  
10                      17430     1560                  25093  
11                       3952      130                   8832  
12                        372        8                    569  
13                       2914       65                   5406  
14                       1151       10                   1895  
15                       4804      102                   7734  
16                       1326       20                   2697  
17                         92        1                    687  
18                       8388      482                  11244  
19                      59166     5651                 116752  
20                        192        0                    552  
21                         30        1                     44  
22                          1        0        

In [5]:
# drop S.NO column
data = data.drop('S. No.', axis=1)

In [6]:
data.head()

Name of State / UT Active Cases* Cured/Discharged/Migrated*  \
0  Andaman and Nicobar Islands            11                         33   
1               Andhra Pradesh          3340                       3641   
2            Arunachal Pradesh            92                          7   
3                        Assam          1955                       2642   
4                        Bihar          1972                       4926   

  Deaths** Total Confirmed cases*  
0        0                     44  
1       90                   7071  
2        0                     99  
3        8                   4605  
4       44                   6942

In [7]:
#rename column names
data = data.rename(columns={'Name of State / UT':'States'})
data = data.rename(columns=lambda x: re.sub('Total Confirmed cases \(Including .* foreign Nationals\) ',
                                                      'Total Cases',x))
data = data.rename(columns=lambda x: re.sub('Deaths \( more than 70% cases due to comorbidities \)',
                                                      'Deaths', x))
data = data.rename(columns={'Total Confirmed cases*': 'Total Cases'})
data = data.rename(columns={'Deaths**':'Death'})
data = data.rename(columns={'Cured/Discharged/Migrated*':'Recovered'})
data = data.rename(columns={'Deaths':'Death'})
data = data.rename(columns={'Active Cases*':'Active Cases'})

# remove extra characters from 'Name of State/UT' column
data['States'] = data['States'].str.replace('#', '')

# remove extra characters from 'Name of State/UT' column
data['Death'] = data['Death'].str.replace('#', '')

# drop active cases colum
data = data.drop('Active Cases', axis=1)

In [8]:
data.head()

States Recovered Death Total Cases
0  Andaman and Nicobar Islands        33     0          44
1               Andhra Pradesh      3641    90        7071
2            Arunachal Pradesh         7     0          99
3                        Assam      2642     8        4605
4                        Bihar      4926    44        6942

In [9]:
#add todays date and create new column
now  = datetime.now()
data['Date'] = now.strftime("%m/%d/%Y") 
data['Date'] = pd.to_datetime(data['Date'], format='%m/%d/%Y')
data.head()

States Recovered Death Total Cases       Date
0  Andaman and Nicobar Islands        33     0          44 2020-06-18
1               Andhra Pradesh      3641    90        7071 2020-06-18
2            Arunachal Pradesh         7     0          99 2020-06-18
3                        Assam      2642     8        4605 2020-06-18
4                        Bihar      4926    44        6942 2020-06-18

In [10]:
#add latitude and longitude
lat = {'Delhi':28.7041, 'Haryana':29.0588, 'Kerala':10.8505, 'Rajasthan':27.0238,
       'Telengana':18.1124, 'Uttar Pradesh':26.8467, 'Ladakh':34.2996, 'Tamil Nadu':11.1271,
       'Andhra Pradesh':15.9129, 'Odisha':20.9517, 'Uttarakhand':30.0668, 'West Bengal':22.9868, 
       'Puducherry': 11.9416, 'Chandigarh': 30.7333, 'Chhattisgarh':21.2787, 
       'Jammu and Kashmir':33.7782, 'Punjab':31.1471, 'Karnataka':15.3173, 
       'Maharashtra':19.7515, 'Gujarat': 22.2587, 
       'Himachal Pradesh': 31.1048, 'Madhya Pradesh': 22.9734, 'Bihar': 25.0961, 'Manipur':24.6637, 
       'Mizoram':23.1645, 'Goa': 15.2993, 'Andaman and Nicobar Islands': 11.7401, 'Assam' : 26.2006, 
       'Jharkhand': 23.6102, 'Arunachal Pradesh': 28.2180, 'Tripura': 23.9408, 'Nagaland': 26.1584, 
       'Meghalaya' : 25.4670}

long = {'Delhi':77.1025, 'Haryana':76.0856, 'Kerala':76.2711, 'Rajasthan':74.2179,
        'Telengana':79.0193, 'Uttar Pradesh':80.9462, 'Ladakh':78.2932, 'Tamil Nadu':78.6569,
        'Andhra Pradesh':79.7400, 'Odisha':85.0985, 'Uttarakhand':79.0193, 'West Bengal':87.8550, 
        'Puducherry': 79.8083, 'Chandigarh': 76.7794, 'Chhattisgarh':81.8661,
        'Jammu and Kashmir':76.5762, 'Punjab':75.3412, 'Karnataka':75.7139, 
        'Maharashtra':75.7139, 'Gujarat': 71.1924, 
        'Himachal Pradesh': 77.1734, 'Madhya Pradesh': 78.6569, 'Bihar': 85.3131, 'Manipur':93.9063, 
        'Mizoram':92.9376, 'Goa': 74.1240, 'Andaman and Nicobar Islands': 92.6586, 'Assam' : 92.9376, 
        'Jharkhand': 85.2799, 'Arunachal Pradesh': 94.7278, 'Tripura': 91.9882, 'Nagaland': 94.5624,
        'Meghalaya' : 91.3662}

data['Latitude'] = data['States'].map(lat)
data['Longitude'] = data['States'].map(long)

In [11]:
data.head()

States Recovered Death Total Cases       Date  \
0  Andaman and Nicobar Islands        33     0          44 2020-06-18   
1               Andhra Pradesh      3641    90        7071 2020-06-18   
2            Arunachal Pradesh         7     0          99 2020-06-18   
3                        Assam      2642     8        4605 2020-06-18   
4                        Bihar      4926    44        6942 2020-06-18   

   Latitude  Longitude  
0   11.7401    92.6586  
1   15.9129    79.7400  
2   28.2180    94.7278  
3   26.2006    92.9376  
4   25.0961    85.3131

In [12]:
my_folder = './.daily_update/'
if not os.path.exists(my_folder):
    os.makedirs(my_folder)

file_name = my_folder + now.strftime("%Y_%m_%d") + '.csv'

In [13]:
data.to_csv(file_name, index=False)

In [14]:
csv_files = glob.glob(my_folder + '*.csv')

all_data = []
for i in csv_files:
    temp = pd.read_csv(i)
    all_data.append(temp)
    

In [15]:
final_data = pd.concat(all_data, ignore_index=True).sort_values(['Date'], ascending=True).reset_index(drop=True)
final_data['Date'] = pd.to_datetime(final_data['Date'])
#final_data.drop(final_data.index[[1005]])
final_data['Active'] = final_data['Total Cases'] - final_data['Recovered'] - final_data['Death']
final_data = final_data.sort_values(['Date', 'States']).reset_index(drop=True)
final_data

/Users/durgesh/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


Date  Death  Latitude  Longitude  Recovered  \
0    2020-04-21      0   11.7401    92.6586         11   
1    2020-04-21     22   15.9129    79.7400         96   
2    2020-04-21      0   28.2180    94.7278          1   
3    2020-04-21      1   26.2006    92.9376         19   
4    2020-04-21      2   25.0961    85.3131         42   
5    2020-04-21      0   30.7333    76.7794         13   
6    2020-04-21      0   21.2787    81.8661         25   
7    2020-04-21     47   28.7041    77.1025        431   
8    2020-04-21      0   15.2993    74.1240          7   
9    2020-04-21     77   22.2587    71.1924        131   
10   2020-04-21      3   29.0588    76.0856        127   
11   2020-04-21      1   31.1048    77.1734         16   
12   2020-04-21      5   33.7782    76.5762         71   
13   2020-04-21      2   23.6102    85.2799          0   
14   2020-04-21     17   15.3173    75.7139        114   
15   2020-04-21      3   10.8505    76.2711        291   
16   2020-04-21      0   34.2996    78.2932         14   
17   2020-04-21     76   22.9734    78.6569        127   
18   2020-04-21    232   19.7515    75.7139        572   
19   2020-04-21      0   24.6637    93.9063          2   
20   2020-04-21      1   25.4670    91.3662          0   
21   2020-04-21      0   23.1645    92.9376          0   
22   2020-04-21      0   26.1584    94.5624          0   
23   2020-04-21      1   20.9517    85.0985         24   
24   2020-04-21      0   11.9416    79.8083          3   
25   2020-04-21     16   31.1471    75.3412         39   
26   2020-04-21     25   27.0238    74.2179        205   
27   2020-04-21     17   11.1271    78.6569        457   
28   2020-04-21     23   18.1124    79.0193        190   
29   2020-04-21      0   23.9408    91.9882          1   
...         ...    ...       ...        ...        ...   
1874 2020-06-18      6   30.7333    76.7794        303   
1875 2020-06-18     10   21.2787    81.8661       1180   
1876 2020-06-18      0       NaN        NaN         12   
1877 2020-06-18   1904   28.7041    77.1025      17457   
1878 2020-06-18      0   15.2993    74.1240         96   
1879 2020-06-18   1560   22.2587    71.1924      17430   
1880 2020-06-18    130   29.0588    76.0856       3952   
1881 2020-06-18      8   31.1048    77.1734        372   
1882 2020-06-18     65   33.7782    76.5762       2914   
1883 2020-06-18     10   23.6102    85.2799       1151   
1884 2020-06-18    102   15.3173    75.7139       4804   
1885 2020-06-18     20   10.8505    76.2711       1326   
1886 2020-06-18      1   34.2996    78.2932         92   
1887 2020-06-18    482   22.9734    78.6569       8388   
1888 2020-06-18   5651   19.7515    75.7139      59166   
1889 2020-06-18      0   24.6637    93.9063        192   
1890 2020-06-18      1   25.4670    91.3662         30   
1891 2020-06-18      0   23.1645    92.9376          1   
1892 2020-06-18      0   26.1584    94.5624        103   
1893 2020-06-18     11   20.9517    85.0985       3047   
1894 2020-06-18      7   11.9416    79.8083        109   
1895 2020-06-18     78   31.1471    75.3412       2538   
1896 2020-06-18    313   27.0238    74.2179      10467   
1897 2020-06-18      0       NaN        NaN          4   
1898 2020-06-18    576   11.1271    78.6569      27624   
1899 2020-06-18    192       NaN        NaN       3071   
1900 2020-06-18      1   23.9408    91.9882        556   
1901 2020-06-18    435   26.8467    80.9462       8904   
1902 2020-06-18     26   30.0668    79.0193       1254   
1903 2020-06-18    506   22.9868    87.8550       6533   

                                        States  Total Cases  Active  
0                  Andaman and Nicobar Islands           16       5  
1                               Andhra Pradesh          757     639  
2                            Arunachal Pradesh            1       0  
3                                        Assam           35      15  
4                                        Bihar          114     

In [16]:
final_data.to_csv('COVID19INDIA.csv', index=False)